In [12]:
import mido
from miditime import MIDITime
import pandas as pd
import random
from mido import MidiFile, MidiTrack, Message

In [2]:
data = pd.read_csv('sky_distribution.csv')
data.columns = data.columns.str.strip()


In [8]:
mymidi = MIDITime(120, 'dawn_chorus.mid', 5, 5, 1)

In [9]:
data['beat'] = data['azimuth[deg]'] / max(data['azimuth[deg]']) * 100

In [10]:
def scale_radiance_to_pitch(radiance):
    min_r, max_r = data['radiance[W/m2]'].min(), data['radiance[W/m2]'].max()
    pct = (radiance - min_r) / (max_r - min_r)
    pitch = 60 + int(pct * 12) + random.choice([-1, 0, 1])  # slight randomness
    return min(max(pitch, 48), 84)

def scale_radiance_to_velocity(radiance):
    min_r, max_r = data['radiance[W/m2]'].min(), data['radiance[W/m2]'].max()
    pct = (radiance - min_r) / (max_r - min_r)
    velocity = int(50 + pct * 77)
    return min(max(velocity, 30), 127)

def scale_radiance_to_modwheel(radiance):
    min_r, max_r = data['radiance[W/m2]'].min(), data['radiance[W/m2]'].max()
    pct = (radiance - min_r) / (max_r - min_r)
    return int(pct * 127)  # MIDI CC range


In [ ]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for index, row in data.iterrows():
    time = int(row['beat'] * 480 / 120)  # convert beats to ticks assuming 480 ticks/beat

    pitch = scale_radiance_to_pitch(row['radiance[W/m2]'])
    velocity = scale_radiance_to_velocity(row['radiance[W/m2]'])
    mod_val = scale_radiance_to_modwheel(row['radiance[W/m2]'])

    # Control change - mod wheel (CC 1)
    track.append(Message('control_change', control=1, value=mod_val, time=0))

    # Optional pitch bend for birdy warble
    if random.random() > 0.8:
        track.append(Message('pitchwheel', pitch=random.randint(-2000, 2000), time=0))

    # Note on
    track.append(Message('note_on', note=pitch, velocity=velocity, time=0))

    # Note off after a short random delay (add variation)
    duration_ticks = int(random.uniform(60, 180))  # 1/8 to 3/8 note approx.
    track.append(Message('note_off', note=pitch, velocity=velocity, time=duration_ticks))

In [14]:
mid.save('dawn_chorus_modulated.mid')